task name: Implement ApplyONNXModelWithInMemoryImages
task description: Implement the ApplyONNXModelWithInMemoryImages example using the provided C# code

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext();

In [ ]:
public class ImageDataPoint
{
    public byte[] Image { get; set; }
    public float[] Scores { get; set; }

    public ImageDataPoint(byte red, byte green, byte blue)
    {
        byte[] imageData = new byte[224 * 224 * 4]; // 4 for the red, green, blue and alpha colors
        for (int i = 0; i < imageData.Length; i += 4)
        {
            // Fill the buffer with the Bgra32 format
            imageData[i] = blue;
            imageData[i + 1] = green;
            imageData[i + 2] = red;
            imageData[i + 3] = 255;
        }

        Image = imageData;
    }
}

var dataPoints = new ImageDataPoint[]
{
    new ImageDataPoint(red: 255, green: 0, blue: 0), // Red color
    new ImageDataPoint(red: 0, green: 128, blue: 0)  // Green color
};
var dataView = mlContext.Data.LoadFromEnumerable(dataPoints);

In [ ]:
var pipeline = mlContext.Transforms.Conversion.ConvertType("Features", "Image", DataKind.Single)
    .Append(mlContext.Transforms.Concatenate("Features", "Features"))
    .AppendCacheCheckpoint(mlContext);

var model = pipeline.Fit(dataView);
var onnx = model.Transform(dataView);

In [ ]:
public class ImageDataPoint
{
    public byte[] Image { get; set; }
    public float[] Scores { get; set; }
    public ImageDataPoint() { }
    public ImageDataPoint(byte red, byte green, byte blue)
    {
        byte[] imageData = new byte[224 * 224 * 4]; // 4 for the red, green, blue and alpha colors
        for (int i = 0; i < imageData.Length; i += 4)
        {
            // Fill the buffer with the Bgra32 format
            imageData[i] = blue;
            imageData[i + 1] = green;
            imageData[i + 2] = red;
            imageData[i + 3] = 255;
        }

        Image = imageData;
    }
}

var transformedDataPoints = mlContext.Data.CreateEnumerable<
    ImageDataPoint>(onnx, false).ToList();
if (transformedDataPoints.Count > 0)
{
    foreach (var dataPoint in transformedDataPoints)
    {
        if (dataPoint.Scores != null && dataPoint.Scores.Length > 0)
        {
            var firstClassProb = dataPoint.Scores.First();
            var lastClassProb = dataPoint.Scores.Last();
            Console.WriteLine("The probability of being the first class is " +
                (firstClassProb * 100) + "%.");

            Console.WriteLine($"The probability of being the last class is " +
                (lastClassProb * 100) + "%.");
        }
        else
        {
            Console.WriteLine("No scores were found for this data point.");
        }
    }
}
else
{
    Console.WriteLine("No data points were transformed.");
}